Done by Mahesh Babu

Task 1: Exploratory Data Analysis (EDA) and Business Insights
1. Perform EDA on the provided dataset.
2. Derive at least 5 business insights from the EDA.
3. Write these insights in short point-wise sentences (maximum 100 words per insight).

In [48]:
import pandas as pd
import numpy as np
import datetime
import plotly.express as px
import plotly.graph_objs as go

In [50]:
# Load the datasets
customers = pd.read_csv('Customers.csv')
products = pd.read_csv('Products.csv')
transactions = pd.read_csv('Transactions.csv')

In [51]:
# Data Preprocessing
# Convert date columns to datetime
customers['SignupDate'] = pd.to_datetime(customers['SignupDate'])
transactions['TransactionDate'] = pd.to_datetime(transactions['TransactionDate'])

In [52]:
# Merge datasets
merged_data = transactions.merge(customers, on='CustomerID')
merged_data = merged_data.merge(products, on='ProductID')

In [53]:
# 1. Customer Acquisition Analysis
def customer_acquisition_analysis(customers):
    # Signup trend over time
    customers['SignupYear'] = customers['SignupDate'].dt.year
    customers['SignupMonth'] = customers['SignupDate'].dt.month

    signup_trend = customers.groupby(['SignupYear', 'SignupMonth']).size().reset_index(name='NewCustomers')
    signup_trend['YearMonth'] = signup_trend['SignupYear'].astype(str) + '-' + signup_trend['SignupMonth'].astype(str)

    # Using Plotly for visualization
    fig = px.line(signup_trend, x='YearMonth', y='NewCustomers',
                  title='Customer Signup Trend',
                  labels={'YearMonth': 'Year-Month', 'NewCustomers': 'Number of New Customers'})
    fig.write_html("customer_signup_trend.html")

    return signup_trend


In [54]:
# 2. Regional Sales Distribution
def regional_sales_distribution(merged_data):
    region_sales = merged_data.groupby('Region')['TotalValue'].agg(['sum', 'mean', 'count']).reset_index()
    region_sales.columns = ['Region', 'TotalSales', 'AvgSaleValue', 'NumberOfTransactions']
    region_sales = region_sales.sort_values('TotalSales', ascending=False)

    # Using Plotly for bar chart
    fig = px.bar(region_sales, x='Region', y='TotalSales',
                 title='Total Sales by Region',
                 labels={'TotalSales': 'Total Sales (USD)', 'Region': 'Region'})
    fig.write_html("regional_sales.html")

    return region_sales

In [55]:
# 3. Product Category Performance
def product_category_performance(merged_data):
    category_sales = merged_data.groupby('Category').agg({
        'TotalValue': ['sum', 'mean'],
        'Quantity': ['sum', 'mean'],
        'TransactionID': 'count'
    }).reset_index()
    category_sales.columns = ['Category', 'TotalSales', 'AvgSaleValue', 'TotalQuantitySold', 'AvgQuantityPerTransaction', 'NumberOfTransactions']
    category_sales = category_sales.sort_values('TotalSales', ascending=False)

    # Using Plotly for pie chart
    fig = px.pie(category_sales, values='TotalSales', names='Category',
                 title='Sales Distribution by Product Category')
    fig.write_html("category_sales.html")

    return category_sales

In [56]:
# 4. Customer Retention and Repeat Purchase Analysis
def customer_retention_analysis(merged_data):
    # Count transactions per customer
    customer_transaction_counts = merged_data.groupby('CustomerID')['TransactionID'].count().reset_index()
    customer_transaction_counts.columns = ['CustomerID', 'TransactionCount']

    # Using Plotly for histogram
    fig = px.histogram(customer_transaction_counts, x='TransactionCount',
                       title='Distribution of Transaction Counts per Customer',
                       labels={'TransactionCount': 'Number of Transactions', 'count': 'Number of Customers'})
    fig.write_html("customer_transaction_distribution.html")

    # Repeat customer rate
    total_customers = merged_data['CustomerID'].nunique()
    repeat_customers = customer_transaction_counts[customer_transaction_counts['TransactionCount'] > 1]['CustomerID'].count()
    repeat_customer_rate = repeat_customers / total_customers * 100

    return {
        'TotalCustomers': total_customers,
        'RepeatCustomers': repeat_customers,
        'RepeatCustomerRate': repeat_customer_rate
    }

In [57]:
# 5. Seasonal Sales Trends
def seasonal_sales_trends(merged_data):
    merged_data['SaleMonth'] = merged_data['TransactionDate'].dt.month
    monthly_sales = merged_data.groupby('SaleMonth')['TotalValue'].sum().reset_index()

    # Using Plotly for line chart
    fig = px.line(monthly_sales, x='SaleMonth', y='TotalValue',
                  title='Monthly Sales Trend',
                  labels={'SaleMonth': 'Month', 'TotalValue': 'Total Sales (USD)'})
    fig.write_html("monthly_sales_trend.html")

    return monthly_sales


In [58]:
# Run analyses
signup_trend = customer_acquisition_analysis(customers)
region_sales = regional_sales_distribution(merged_data)
category_sales = product_category_performance(merged_data)
retention_analysis = customer_retention_analysis(merged_data)
monthly_sales = seasonal_sales_trends(merged_data)

In [59]:
# Print key insights
print("Business Insights Summary:")
print("\n1. Customer Acquisition:")
print(signup_trend.nlargest(3, 'NewCustomers'))

print("\n2. Regional Sales Performance:")
print(region_sales)

print("\n3. Product Category Performance:")
print(category_sales)

print("\n4. Customer Retention:")
print(f"Repeat Customer Rate: {retention_analysis['RepeatCustomerRate']:.2f}%")

print("\n5. Seasonal Sales:")
print(monthly_sales)

Business Insights Summary:

1. Customer Acquisition:
    SignupYear  SignupMonth  NewCustomers YearMonth
32        2024            9            11    2024-9
34        2024           11            11   2024-11
27        2024            4            10    2024-4

2. Regional Sales Performance:
          Region  TotalSales  AvgSaleValue  NumberOfTransactions
3  South America   219352.56    721.554474                   304
1         Europe   166254.63    710.489872                   234
2  North America   152313.40    624.235246                   244
0           Asia   152074.97    697.591606                   218

3. Product Category Performance:
      Category  TotalSales  AvgSaleValue  TotalQuantitySold  \
0        Books   192147.47    711.657296                681   
2  Electronics   180783.50    711.746063                627   
1     Clothing   166170.66    728.818684                590   
3   Home Decor   150893.93    608.443266                639   

   AvgQuantityPerTransaction  Nu